In [0]:
import os, warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool) # dropout 유무 판단

In [0]:
# 02. CNN 계층 구성
# W1: 가중치
# W1 [3 3 1 32]
# [3,3] : 커널크기
# 1 : 입력값 채널수
# 32 : 필터의 개수 (커널의 개수)
# L1 : 계층
# (?, 28, 28, 32)
# ? : 데이터 개수는 확정되지 않음.
# 28, 28 : 커널을 통해 만들어진 이미지 픽셀
# 32 : 커널(필터)의 개수
# Pool : 플링 계층
# (?, 14, 14, 32)
# ? : 데이터 개수가 정해져 있지 않음.
# 14, 14 : 커널을 통해 만들어진 이미지 픽셀
# 32 : 커널(필터)의 개수

In [6]:
# W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'conv2d/BiasAdd:0' shape=(?, 26, 26, 32) dtype=float32>

In [7]:
# L1 = tf.nn.relu(L1) # 활성화 함수
# L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1

Instructions for updating:
Use keras.layers.MaxPooling2D instead.


<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(?, 13, 13, 32) dtype=float32>

In [8]:
L1 = tf.layers.dropout(L1, 0.7, is_training)
L1

Instructions for updating:
Use keras.layers.dropout instead.


<tf.Tensor 'dropout/Identity:0' shape=(?, 13, 13, 32) dtype=float32>

In [9]:
# W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
# L2 = tf.nn.relu(L2)
# L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.layers.conv2d(L1, 64, [3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

In [10]:
# W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
# L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
# L3 = tf.matmul(L3, W3)
# L3 = tf.nn.relu(L3)
# L3 = tf.nn.dropout(L3, keep_prob)
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [11]:
model = tf.layers.dense(L3, 10, activation = None)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [13]:
%%time
#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(10):
  total_cost = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    _, cost_val = sess.run([optimizer, cost],
    feed_dict={X: batch_xs,
    Y: batch_ys,
    is_training: True})

    total_cost += cost_val
  print('Epoch:', '%04d' % (epoch + 1),
  'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.168
Epoch: 0002 Avg. cost = 0.048
Epoch: 0003 Avg. cost = 0.030
Epoch: 0004 Avg. cost = 0.021
Epoch: 0005 Avg. cost = 0.013
Epoch: 0006 Avg. cost = 0.013
Epoch: 0007 Avg. cost = 0.011
Epoch: 0008 Avg. cost = 0.010
Epoch: 0009 Avg. cost = 0.008
Epoch: 0010 Avg. cost = 0.006
최적화 완료!
CPU times: user 25 s, sys: 6.31 s, total: 31.3 s
Wall time: 36.8 s


In [14]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
Y: mnist.test.labels,
is_training: False}))

정확도: 0.988
